In [ ]:
# File is to define the metrics to evaluate predictions on

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import imageio
import os
import time

In [ ]:
def calc_dists(lines1, lines2, dist_type):
    x1s = lines1[:, 2]
    y1s = lines1[:, 3]
    x2s = lines2[:, 2]
    y2s = lines2[:, 3]
    if dist_type == "L2":
        return np.mean(np.sqrt(np.square(y2s - y1s) + np.square(x2s - x1s)))
    return np.mean(np.abs(y2s-y1s) + np.abs(x2s-x1s))
    
def extract_trajectory(data, ped_id):
    return data[data[:, 1] == ped_id, :]

def overall_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)
    traj2 = extract_trajectory(pred, ped_id)
    dist = calc_dists(traj1, traj2, dist_type) / 12
    return dist

def predicted_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)[8:, :] # first 8 are ignored since those are given
    traj2 = extract_trajectory(pred, ped_id)[8:, :]
    dist = calc_dists(traj1, traj2, dist_type) / 12.
    return dist

def final_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)[-1, :].reshape((1,4)) # only use last one
    traj2 = extract_trajectory(pred, ped_id)[-1, :].reshape((1,4))
    dist = calc_dists(traj1, traj2, dist_type)
    return dist

def get_mean_fn(mean_type):
    if mean_type == "overall":
        return overall_mean
    if mean_type == "predicted":
        return predicted_mean
    return final_mean

# mean_type = overall, predicted, final
# dist_type = L2, L1
def prediction_error(gt, pred, mean_type, dist_type, lin, linmap):
    peds = np.sort(np.unique(gt[:, 1]))
    npeds = peds.shape[0]
    dist = 0.0
    for i in range(npeds):
        if lin == 0 or (lin == 1 and linmap[peds[i]]) or (lin == 2 and not linmap[peds[i]]):
            dist += get_mean_fn(mean_type)(gt, pred, peds[i], dist_type)
    dist /= npeds
    return dist

num_metrics = 4 # 6

def file_error(gt, pred, lin, linmap):
    return [#prediction_error(gt, pred, "overall", "L2"), 
            prediction_error(gt, pred, "predicted", "L2", lin, linmap), 
            prediction_error(gt, pred, "final", "L2", lin, linmap),
            #prediction_error(gt, pred, "overall", "L1"),
            prediction_error(gt, pred, "predicted", "L1", lin, linmap),
            prediction_error(gt, pred, "final", "L1", lin, linmap)]

def print_leaderboard(gt_dir, predict_dir, lin):
    datasets = {}
    datasets[".overall"] = np.zeros((num_metrics,))
    overallcount = 0.0
    for folder in os.listdir(gt_dir):
        if "." in folder:
            continue
        datasets[folder] = {}
        datasets[folder][".err"] = np.zeros((num_metrics,))
        count = 0.0
        for fname in os.listdir(gt_dir + folder):
            if not fname.endswith(".txt"):
                continue
            #print "Processing", fname
            gtfile = gt_dir + folder + "/" + fname
            predfile = predict_dir + folder + "/" + fname
            gt = np.loadtxt(gtfile)
            pred = np.loadtxt(predfile)
            #print gt.shape
            #print pred.shape
            fname2 = gt_dir + folder + "/" + fname
            pname = fname2[fname2.rfind("/")+1:fname2.rfind(".")]
            datasets[folder][fname] = file_error(gt, pred, lin, linear_map[pname])
            datasets[folder][".err"] += np.asarray(datasets[folder][fname])
            datasets[".overall"] += np.asarray(datasets[folder][fname])
            count += 1
            overallcount += 1
        datasets[folder][".err"] /= count
    datasets[".overall"] /= overallcount
    return datasets

In [ ]:
def predictions_leaderboard(gtfile, predictfile, lin):
    if lin == 0:
        print "All Sequences"
    elif lin == 1:
        print "Linear Sequences"
    else:
        print "Non-Linear Sequences"
    datasets = print_leaderboard(gtfile, predictfile, lin)
    
#     for folder in datasets:
#         print folder
#         for fname in datasets[folder]:
#             print fname
    
    #print datasets
    formatstr = '{: <25}{: <12}{: <12}{: <12}{: <12}'
    #formatstr = '{: <25}{: <12}{: <12}{: <12}{: <12}{: <12}{: <12}'
#     print formatstr.format("File", "O-L2", "P-L2", "F-L2", "O-L1", "P-L1", "F-L1")
    print formatstr.format("File", "P-L2", "F-L2", "P-L1", "F-L1")

    for folder in datasets:
        if folder == ".overall":
            es = datasets[folder]
#             print formatstr.format("Overall", round(es[0], 5), round(es[1], 5), round(es[2], 5), round(es[3], 5), round(es[4], 5), round(es[5], 5))
            print formatstr.format("Overall", round(es[0], 5), round(es[1], 5), round(es[2], 5), round(es[3], 5))
            continue
        
        es = datasets[folder][".err"]
        print formatstr.format(folder + "/", round(es[0], 5), round(es[1], 5), round(es[2], 5), round(es[3], 5))
        for fname in datasets[folder]:
            if fname == ".err":
                continue
#             print "\t",fname
            ffname = "    " + fname #"    " + fname if fname != ".err" else folder + "/"
            es = datasets[folder][fname]
#             print ffname
            print formatstr.format(ffname, round(es[0], 5), round(es[1], 5), round(es[2], 5), round(es[3], 5))

#             print formatstr.format(ffname, round(es[0], 5), round(es[1], 5), round(es[2], 5), round(es[3], 5), round(es[4], 5), round(es[5], 5))


In [ ]:
predictions_leaderboard("../data/challenges/1/gt/", "../data/challenges/1/predict_sf_ewap/", lin=2) # 0 is both, 1 is just linear, 2 is just non-linear

In [ ]:
predictions_leaderboard("../data/challenges/1/gt/", "../data/challenges/1/predict_sf_attr/", lin=2) # 0 is both, 1 is just linear, 2 is just non-linear

In [ ]:
predictions_leaderboard("../data/challenges/1/gt/", "../data/challenges/1/predict_linear/", lin=2) # 0 is both, 1 is just linear, 2 is just non-linear

In [ ]:
predictions_leaderboard("../data/challenges/1/gt/", "../data/challenges/1/predict_igp/", lin=2) # 0 is both, 1 is just linear, 2 is just non-linear

In [ ]:
# print file_error(
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"),
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"))

# print file_error(
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"),
#     np.loadtxt("../data/challenges/1/predict_lin/crowds/crowds_zara01.txt"))

folders = ["biwi", "crowds", "crowds"]
fnames = ["biwi_eth", "uni_examples", "crowds_zara01"]

for lin in range(3):
    if lin == 0:
        print "All sequences"
    elif lin == 1:
        print "Just Linear sequences"
    elif lin == 2:
        print "Just non-linear sequences"
    else:
        print "NOT SURE"
    for i in range(len(folders)):

        print fnames[i]
        folder = folders[i]
        fname = fnames[i]

        print "SF EWAP"
        print file_error(
            np.loadtxt("../data/challenges/1/gt/" + folder + "/" + fname + ".txt"),
            np.loadtxt("../data/challenges/1/predict_sf_ewap/" + folder + "/" + fname + ".txt"),
            lin,
            linear_map[fname]
        )

        print "LINEAR"
        print file_error(
            np.loadtxt("../data/challenges/1/gt/" + folder + "/" + fname + ".txt"),
            np.loadtxt("../data/challenges/1/predict_linear/" + folder + "/" + fname + ".txt"),
            lin,
            linear_map[fname]
        )
        
        print "SF ATTR"
        print file_error(
            np.loadtxt("../data/challenges/1/gt/" + folder + "/" + fname + ".txt"),
            np.loadtxt("../data/challenges/1/predict_sf_attr/" + folder + "/" + fname + ".txt"),
            lin,
            linear_map[fname]
        )
        
        print "IGP"
        print file_error(
            np.loadtxt("../data/challenges/1/gt/" + folder + "/" + fname + ".txt"),
            np.loadtxt("../data/challenges/1/predict_igp/" + folder + "/" + fname + ".txt"),
            lin,
            linear_map[fname]
        )

        print "NAIVE-LSTM"
        print file_error(
            np.loadtxt("../data/challenges/1/gt/" + folder + "/" + fname + ".txt"),
            np.loadtxt("../data/challenges/1/predict_naive/" + folder + "/" + fname + ".txt"),
            lin,
            linear_map[fname]
        )

In [ ]:
import numpy.polynomial.polynomial as poly

def trajectory_distance(traj):
    total_err = 0.0
    rng = range(20)
    x = traj[:, 2]
    y = traj[:, 3]
    xcoefs = poly.polyfit(rng, x, 1) # fit line for x
    ycoefs = poly.polyfit(rng, y, 1) # fit line for y

    for i in range(0, traj.shape[0]):
        pred_x = xcoefs[0] + i*xcoefs[1]
        pred_y = ycoefs[0] + i*ycoefs[1]
        total_err += np.abs(pred_x - traj[i,2]) + np.abs(pred_y - traj[i,3])
    return total_err

linear_map = {}
threshold = 5.0
gt_dir = "../data/challenges/1/gt/"
for folder in os.listdir(gt_dir):
    if "." in folder:
        continue
    for fname in os.listdir(gt_dir + folder):
        if not fname.endswith(".txt"):
            continue
        fname = gt_dir + folder + "/" + fname
        print "Processing", fname
        pname = fname[fname.rfind("/")+1:fname.rfind(".")]
        if not (pname in linear_map):
            linear_map[pname] = {}
        dists = []
        gt = np.loadtxt(fname)
        peds = np.sort(np.unique(gt[:, 1]))
        npeds = peds.shape[0]
        for i in range(npeds):
            traj = extract_trajectory(gt, peds[i])
            dist = trajectory_distance(traj)
            linear_map[pname][peds[i]] = dist < threshold
            dists.append(dist)

        dists = np.asarray(dists)

        plt.hist(dists, normed=True, bins=30, edgecolor='black')
        plt.title(pname)
        plt.savefig("out/linear/" + pname + ".png")
        plt.show()
        plt.close()

In [ ]:
print linear_map

In [ ]:
def extract_positions(data, frame):
    return data[data[:, 0] == frame, :]

def count_overlap(people, threshold=0.5):
    dists = []
    count = 0
    for i in range(people.shape[0]-1):
        for j in range(i+1, people.shape[0]):
            dist = np.mean(np.sqrt(np.square(people[i,3] - people[j,3]) + np.square(people[i,2] - people[j,2])))
            if dist < threshold:
                count += 1
            dists.append(dist)
    return dists, count

In [ ]:
gt_dir = "../data/challenges/1/gt/"
for folder in os.listdir(gt_dir):
    if "." in folder:
        continue
    for fname in os.listdir(gt_dir + folder):
        if not fname.endswith(".txt"):
            continue
        fname = gt_dir + folder + "/" + fname
        print "Processing", fname

        pname = fname[fname.rfind("/")+1:fname.rfind(".")]
        gt = np.loadtxt(fname)
        frames = np.sort(np.unique(gt[:, 0]))
        nframes = frames.shape[0]

        dists = []
        count = 0
        total = 0
        threshold = 0.5
        for i in range(nframes):
            people = extract_positions(gt, frames[i])
            dist, counts = count_overlap(people, threshold=threshold)
            dists.extend(dist)
            count += counts
        total += len(dists)
        dists = np.asarray(dists)

        plt.hist(dists, normed=True, bins=50, edgecolor='black')
        plt.title(pname)
        plt.savefig("out/collisions/" + pname + ".png")
        plt.show()
        plt.close()

        print "Threshold", threshold, "Count", count, "Total", total, "Percentage", round(count*100./total, 3)
        # print dists

In [ ]:
gt_dir = "../data/challenges/1/predict_gt/"
models = ["gt", "predict_igp", "predict_linear", "predict_sf_ewap", "predict_sf_attr"]
        
for folder in os.listdir(gt_dir):
    if "." in folder:
        continue
    for fname in os.listdir(gt_dir + folder):
        if not fname.endswith(".txt"):
            continue

        for modelname in models:

            fname2 = gt_dir.replace("predict_gt", modelname) + folder + "/" + fname
            pname = fname2[fname2.rfind("/")+1:fname2.rfind(".")]

            gt = np.loadtxt(fname2)
            frames = np.sort(np.unique(gt[:, 0]))
            nframes = frames.shape[0]

            dists = []
            count = 0
            total = 0
            threshold = 0.5
            for i in range(nframes):
                people = extract_positions(gt, frames[i])
                dist, counts = count_overlap(people, threshold=threshold)
                dists.extend(dist)
                count += counts
            total += len(dists)
            dists = np.asarray(dists)

            print modelname, pname, round(count*100./total, 3), count, total
#         print "\tPercentage", round(count*100./total, 3), "Count", count, "Total", total
        # print dists

In [ ]:
# print file_error(
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"),
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"))

# print file_error(
#     np.loadtxt("../data/challenges/1/gt/crowds/crowds_zara01.txt"),
#     np.loadtxt("../data/challenges/1/predict_lin/crowds/crowds_zara01.txt"))

folders = ["biwi", "crowds", "crowds"]
fnames = ["biwi_eth", "uni_examples", "crowds_zara01"]
models = ["gt", "predict_linear", "predict_sf_ewap", "predict_sf_attr", "predict_igp", "predict_naive"]

for i in range(len(folders)):

    print fnames[i]
    folder = folders[i]
    fname = fnames[i]

    for model in models:
    
        gt = np.loadtxt("../data/challenges/1/" + model + "/" + folder + "/" + fname + ".txt")
        frames = np.sort(np.unique(gt[:, 0]))
        nframes = frames.shape[0]
    
        dists = []
        count = 0
        total = 0
        threshold = 0.5
        for i in range(nframes):
            people = extract_positions(gt, frames[i])
            dist, counts = count_overlap(people, threshold=threshold)
            dists.extend(dist)
            count += counts
        total += len(dists)
        dists = np.asarray(dists)

        print "\t", model, pname, round(count*100./total, 3), count, total